In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search,Q,A
from elasticsearch_dsl.query import MultiMatch, Match,MoreLikeThis
from pymongo import MongoClient
import time
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["JobAggregator"]
col = mydb['Log']

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port':9200}])


In [4]:
def create_index(es_object, index_name):
    created = False
    # index settings
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },

        
    }
    print(settings)
    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            print('hi')
            es_object.indices.create(index=index_name, body=settings)
            print('Created Index')
            created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created
def store_record(elastic_object, index_name,record):
    try:
        print(record)
        outcome = elastic_object.index(index=index_name, body=record)
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))

In [40]:
search = Search(using = es,index = "raw_article_2")
search_log = Search(using = es,index = "es_log")

In [49]:
# #cai nay la multi match
# while True:
#     query = input("query:")
#     s = search.query("match",body = query).execute()
#     #multi search
#     multi_match = MultiMatch(query = query,fields = ['title','body','description'])
#     #cai nay cung la multi search
#     q = Q("multi_match",query = query, fields = ['title','body','description'])
#     s_2 = search.query(multi_match).execute()
#     s_3 = search.query(q).execute()
#     print(s_3.to_dict())


In [42]:
create_index(es,"es_log")

{'settings': {'number_of_shards': 1, 'number_of_replicas': 0}}


False

In [52]:
# while True:
query = input("Your query: ")
# Search binh thuong
res_3 = es.search(index="raw_article_2", q=query, size=100, from_=1)
res_4 =  es.search(index="raw_article_2", q=query, size=100, filter_path = ['hits.hits._id','hits.hits.body','hits.hits._source.body'])
#     print(res_4)
#luu tu khoa tim kiem vao mot cai log
store_record(es,"es_log",{"query":query,"timestamp":time.time()})

Your query: hi
{'query': 'hi', 'timestamp': 1620269977.8569744}


In [44]:
#tim xem la tu khoa nao duoc tim kiem nhieu nhat trong query log
s = search
a = A("terms",field = "query")
# sl = search_log.query()
# print(sl.to_dict())
# s = s.query("match",body = "hi").query(MoreLikeThis(like="man", fields=['body', 'body_tokenized']))
# # s = s.source(exclude = ["body"])
# print(s.to_dict())
# t = s.execute()
# t.to_dict()

In [71]:
search_log = Search(using = es,index = "es_log")
query_body = {
  "size": 10,
  "aggs": {
    "group_by_query": {
      "terms": {
        "field": "query.keyword"
      }
    }
  }
}
s = search_log.update_from_dict(query_body)
print(s.to_dict())
t = s.execute()
t.aggregations.group_by_query.to_dict()

{'aggs': {'group_by_query': {'terms': {'field': 'query.keyword'}}}, 'size': 10}


{'doc_count_error_upper_bound': 0,
 'sum_other_doc_count': 18,
 'buckets': [{'key': 'chelsea', 'doc_count': 8},
  {'key': 'hi', 'doc_count': 7},
  {'key': '100 ngày hạ', 'doc_count': 5},
  {'key': 'asus', 'doc_count': 1},
  {'key': 'bánh rán', 'doc_count': 1},
  {'key': 'bất động sản', 'doc_count': 1},
  {'key': 'canon fuji', 'doc_count': 1},
  {'key': 'chuột fulen', 'doc_count': 1},
  {'key': 'coin', 'doc_count': 1},
  {'key': 'corona', 'doc_count': 1}]}

In [38]:
# print(s.to_dict())
#tim nhung tu lien quan voi query
query = input("query: ")
q = Q("match",body = query)
print(q.to_dict())
s1 = search
s1 = s1.query(q)
print(s1.to_dict())
s1.aggs.bucket('my_sample','sampler',shard_size = 100).bucket('keywords',"significant_text",field = "body")
print(s1.to_dict())
resp = s1.execute()
print(resp.to_dict())

query: hi
{'match': {'body': 'hi'}}
{'query': {'match': {'body': 'hi'}}}
{'query': {'match': {'body': 'hi'}}, 'aggs': {'my_sample': {'sampler': {'shard_size': 100}, 'aggs': {'keywords': {'significant_text': {'field': 'body'}}}}}}
{'took': 658, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 176, 'relation': 'eq'}, 'max_score': 8.025035, 'hits': [{'_index': 'raw_article_2', '_type': '_doc', '_id': '2BydNXkBB9o1YLOIXh8C', '_score': 8.025035, '_source': {'scraped': 1571738586, 'thumb_url': 'https://cdn.24h.com.vn/upload/4-2019/images/2019-10-22/1571735031-365-ava-1571732435-width640height480-auto-crop-watermark.jpg', 'description_tokenized': ' Trong khoảnh_khắc vui_thích quá_đà An_Hi cúi người_dùng bộ ngực khủng của mình để sử_dụng cần điểu khiển máy gắp thú ', 'body_tokenized': ' Gắp thú là một trong những trò_chơi thu_hút rất đông các cô gái trẻ Vài ngày_trước một cô gái người Đài_Loan bỗng_nhiên khiến cộng_đồng mạng d